In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pickle


In [2]:
def data_cleanup():
    music_df=pd.read_csv('music_genre.csv')
    music_df=music_df.dropna()
    music_df=music_df[music_df['tempo']!='?']
    music_df=music_df[music_df['artist_name']!='empty_field']
    music_df=music_df[music_df['duration_ms']>0]
    data=music_df.copy().drop(['artist_name','track_name','instance_id','obtained_date','popularity','duration_ms'],axis=1).reset_index(drop=True)
    songs=music_df.copy()[['instance_id','artist_name','track_name','music_genre']].reset_index(drop=True)

    # encoding data
    le=LabelEncoder()
    data['key']=le.fit_transform(data['key'])
    data['mode']=le.fit_transform(data['mode'])
    return data, songs


In [3]:
music_df=pd.read_csv('music_genre.csv')
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50005 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence           50000 non-null  float6

In [4]:
music_df=music_df.dropna()

In [5]:
music_df=music_df[music_df['tempo']!='?']

In [6]:
music_df=music_df[music_df['artist_name']!='empty_field']

In [7]:
music_df=music_df[music_df['duration_ms']>0]

In [8]:
music_df

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.950000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.011800,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.002530,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.909000,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
6,43760.0,Jordan Comolli,Clash,46.0,0.02890,0.572,214408.0,0.803,0.000008,B,0.106,-4.294,Major,0.3510,149.995,4-Apr,0.230,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49999,28408.0,Night Lovell,Barbie Doll,56.0,0.13300,0.849,237667.0,0.660,0.000008,C,0.296,-7.195,Major,0.0516,99.988,4-Apr,0.629,Hip-Hop
50001,43557.0,Roy Woods,Drama (feat. Drake),72.0,0.15700,0.709,251860.0,0.362,0.000000,B,0.109,-9.814,Major,0.0550,122.04299999999999,4-Apr,0.113,Hip-Hop
50002,39767.0,Berner,Lovin' Me (feat. Smiggz),51.0,0.00597,0.693,189483.0,0.763,0.000000,D,0.143,-5.443,Major,0.1460,131.079,4-Apr,0.395,Hip-Hop
50003,57944.0,The-Dream,Shawty Is Da Shit,65.0,0.08310,0.782,262773.0,0.472,0.000000,G,0.106,-5.016,Minor,0.0441,75.88600000000001,4-Apr,0.354,Hip-Hop


In [9]:
data=music_df.copy().drop(['artist_name','track_name','instance_id','obtained_date','popularity','duration_ms'],axis=1).reset_index(drop=True)

In [10]:
songs=music_df.copy()[['instance_id','artist_name','track_name','music_genre']].reset_index(drop=True)

In [11]:
# encoding data
le=LabelEncoder()
data['key']=le.fit_transform(data['key'])

In [12]:
data['mode']=le.fit_transform(data['mode'])

In [13]:
data

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,music_genre
0,0.01270,0.622,0.890,0.950000,5,0.124,-7.043,1,0.0300,115.00200000000001,0.531,Electronic
1,0.00306,0.620,0.755,0.011800,11,0.534,-4.617,0,0.0345,127.994,0.333,Electronic
2,0.02540,0.774,0.700,0.002530,4,0.157,-4.498,0,0.2390,128.014,0.270,Electronic
3,0.00465,0.638,0.587,0.909000,9,0.157,-6.266,0,0.0413,145.036,0.323,Electronic
4,0.02890,0.572,0.803,0.000008,2,0.106,-4.294,0,0.3510,149.995,0.230,Electronic
...,...,...,...,...,...,...,...,...,...,...,...,...
38551,0.13300,0.849,0.660,0.000008,3,0.296,-7.195,0,0.0516,99.988,0.629,Hip-Hop
38552,0.15700,0.709,0.362,0.000000,2,0.109,-9.814,0,0.0550,122.04299999999999,0.113,Hip-Hop
38553,0.00597,0.693,0.763,0.000000,5,0.143,-5.443,0,0.1460,131.079,0.395,Hip-Hop
38554,0.08310,0.782,0.472,0.000000,10,0.106,-5.016,1,0.0441,75.88600000000001,0.354,Hip-Hop


In [14]:
songs

,instance_id,artist_name,track_name,music_genre
0,46652.0,Thievery Corporation,The Shining Path,Electronic
1,30097.0,Dillon Francis,Hurricane,Electronic
2,62177.0,Dubloadz,Nitro,Electronic
3,24907.0,What So Not,Divide & Conquer,Electronic
4,43760.0,Jordan Comolli,Clash,Electronic
...,...,...,...,...
38551,28408.0,Night Lovell,Barbie Doll,Hip-Hop
38552,43557.0,Roy Woods,Drama (feat. Drake),Hip-Hop
38553,39767.0,Berner,Lovin' Me (feat. Smiggz),Hip-Hop
38554,57944.0,The-Dream,Shawty Is Da Shit,Hip-Hop


In [15]:

def search_artist(songs):
    while True:
        print('-----------------------------------------------------------------')
        artist=input('Which artist would you like to search a song for ?')
        print('-----------------------------------------------------------------')
        if artist in songs['artist_name'].to_list():
            print(f'You have selected {artist}:')
            # searching for artist
            selected_artist=songs.loc[songs['artist_name']==artist]
            #list songs
            print('-----------------------------------------------------------------')
            print('Here are their songs on file')
            print(selected_artist['track_name'])
            print('-----------------------------------------------------------------')
            return selected_artist, artist

        else: 
            print(f'{artist} not in the database.')
            print('please try something else.')

def song_to_search(songs,data):
    selected_artist,artist=search_artist(songs)
    # selecting song and getting its unique id
    song_name=input('Which song would you like to search ?')
    while True:
    
        if song_name.isdigit():
            id=selected_artist.loc[int(song_name)]['instance_id']
            
            song_type=songs.loc[int(song_name)]['music_genre']
            #isolating song_type
            filtered_songs=songs.loc[songs['music_genre']==song_type].reset_index(drop=True)
            # getting the song's index number
            song_idx=filtered_songs.loc[filtered_songs['instance_id']==id].index
            song_idx=song_idx[0]
            new_song_name=selected_artist.loc[int(song_name)]['track_name']
            print('-----------------------------------------------------------------')
            print(f'selected song: {artist} : {new_song_name} : {song_type}')
            print('-----------------------------------------------------------------')
            filtered_data=data.loc[data['music_genre']==song_type].reset_index(drop=True)
            return song_idx, filtered_songs, filtered_data
        else:
            id=selected_artist.loc[selected_artist['track_name']==song_name]['instance_id'].reset_index(drop=True)
            id=id[0]
            # getting song_type
            song_type=songs.loc[songs['instance_id']==id]['music_genre'].reset_index(drop=True)
            song_type=song_type[0]
            #isolating song_type
            filtered_songs=songs.loc[songs['music_genre']==song_type].reset_index(drop=True)
            # getting the song's index number
            song_idx=filtered_songs.loc[filtered_songs['instance_id']==id].index
            song_idx=song_idx[0]
            print('-----------------------------------------------------------------')
            print(f'selected song: {artist} : {song_name} : {song_type}')
            print('-----------------------------------------------------------------')
            filtered_data=data.loc[data['music_genre']==song_type].reset_index(drop=True)
            return song_idx, filtered_songs, filtered_data
            


In [16]:
def search_engine(songs,data):
    song_idx, filtered_songs, filtered_data=song_to_search(songs,data)
    filtered_data=filtered_data.drop('music_genre',axis=1)
    scaler=StandardScaler()
    data_scaled=scaler.fit_transform(filtered_data)     

    #compute similarities between songs
    cosine_sim =cosine_similarity(data_scaled,data_scaled)

    #converting matrix to dataframe
    cosine_sim=pd.DataFrame(cosine_sim)

    # iterating through the similarity scores to find closest to the song selected using index
    rec=cosine_sim[song_idx]
    score=rec.sort_values(ascending=False)
    similar=filtered_songs.iloc[score.index]
    best_10=similar[1:10].reset_index(drop=True)
    return best_10

In [17]:
# selected_artist,artist=search_artist(songs)
# # selecting song and getting its unique id
# song_name=input('Which song would you like to search ?')
# while True:

#     if song_name.isdigit():
        
#         id=selected_artist.loc[int(song_name)]['instance_id']
        
#         song_type=songs.loc[int(song_name)]['music_genre']
#         #isolating song_type
#         filtered_songs=songs.loc[songs['music_genre']==song_type].reset_index(drop=True)
#         # getting the song's index number
#         song_idx=filtered_songs.loc[filtered_songs['instance_id']==id].index
#         song_idx=song_idx[0]
#         new_song_name=selected_artist.loc[int(song_name)]['track_name']
#         print('-----------------------------------------------------------------')
#         print(f'selected song: {artist} : {new_song_name} : {song_type}')
#         print('-----------------------------------------------------------------')
#         filtered_data=data.loc[data['music_genre']==song_type].reset_index()
#         break
#     else:
#         print('no')

In [18]:
results=search_engine(songs,data)

-----------------------------------------------------------------
-----------------------------------------------------------------
27047 not in the database.
please try something else.
-----------------------------------------------------------------
-----------------------------------------------------------------
You have selected Muse:
-----------------------------------------------------------------
Here are their songs on file
27047                                              Reapers
27208                                           Resistance
27264                                             Assassin
27367                                            Starlight
27378                                                Mercy
27409                                       Break it to Me
27470                                             Dig Down
27868                                        The Dark Side
28085                                  Time Is Running Out
28362                           

In [19]:
print(results)

   instance_id                artist_name  \
0      45473.0                  Twin Wild   
1      71748.0                  Foreigner   
2      47589.0           System Of A Down   
3      73271.0  Catfish and the Bottlemen   
4      68532.0                   Deftones   
5      61900.0             Arctic Monkeys   
6      53807.0                 Zayde Wølf   
7      30761.0      Francesca Battistelli   
8      37140.0                  Pearl Jam   

                                  track_name music_genre  
0                                Willow Tree        Rock  
1  Feels Like the First Time - 2008 Remaster        Rock  
2                                   Toxicity        Rock  
3                                   Longshot        Rock  
4               Back To School - Mini Maggit        Rock  
5                              Black Treacle        Rock  
6                                  New Blood        Rock  
7                           He Knows My Name        Rock  
8                 